In [37]:
import sys
import rdflib
import matplotlib
import pandas as pd
import scipy.stats as ss
from IPython.core.display import display, HTML
from SPARQLWrapper import SPARQLWrapper, JSON, XML, N3
from decimal import Decimal
import visJS2jupyter.visJS_module
import visJS2jupyter.visualizations
import networkx as nx

import random

def getPrefixDec(prefixes):
    l = ""
    for k,v in prefixes.items():
        l = l + "PREFIX " + k + ": <" + v + ">" + "\r\n"
    return l

def getValuesDec(entities):
    l = ""
    for i in entities:
        if(i[0:4] == "http"):
            l = l + "(<" + i + ">) "
        else:
            l = l + "(" + i + ") "
    return l

def getOrphanetPhenotypes(disease, endpoint, prefixes, entities = "", predicates = "", triplepattern = ""):
    prefixDec = getPrefixDec(prefixes)
    eValuesDec = " VALUES (?s) {" + getValuesDec(entities) + "}"
    pValuesDec = " VALUES (?p) {" + getValuesDec(predicates) + "}"
    sparql = SPARQLWrapper(endpoint)
    query = prefixDec + """ 

CONSTRUCT 
{
    ?d <http://bio2rdf.org/has-phenotype> ?sign .
    ?sign dct:title ?signLabel .
}
{ 
  ?d ov:sign-freq ?sf .
  FILTER (?d = """ + disease + """)
  ?sf ov:frequency ?f .
  ?f dct:title "Very frequent"@en .
  ?sf ov:sign ?sign .
  ?sign dct:title ?signLabel .

}
    """
   
    sparql.setQuery(query)
    sparql.setReturnFormat(XML)
    results = sparql.query().convert()
    
    #print(results.serialize(format='n3').decode())
    
   # print(results)
   # g = rdflib.Graph()
   # g.parse(data=results, format="xml")
   # print(g.serialize(format='n3'))

    return results

localhost = "http://10.106.2.140:9999/blazegraph/sparql"
bio2rdf   = "http://bio2rdf.org/sparql"

prefixes = {"dct":"http://purl.org/dc/terms/", "omim":"http://bio2rdf.org/omim:", 
            "ov":"http://bio2rdf.org/orphanet_vocabulary:", "orphanet":"http://bio2rdf.org/orphanet:" }
disease = "omim:511775"
disease = "orphanet:2331"

phenotypes = getOrphanetPhenotypes(disease, localhost, prefixes)





In [36]:
G = nx.connected_watts_strogatz_graph(30, 5, 0.2)
nodes = list(G.nodes()) # must cast to list to maintain compatibility between nx 1.11 and 2.0
edges = list(G.edges()) # will return an "EdgeView" object in nx 2.0

# define the initial positions of the nodes using networkx's spring_layout function, and add to the nodes_dict.
pos = nx.spring_layout(G)

nodes_dict = [{"id":n,
              "x":pos[n][0]*300,
              "y":pos[n][1]*300} for n in nodes]

node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

edges_dict = [{"source":node_map[edges[i][0]], "target":node_map[edges[i][1]], 
              "title":'test'} for i in range(len(edges))]

visJS2jupyter.visJS_module.visjs_network(nodes_dict, edges_dict)



In [21]:
from pyvis.network import Network
net = Network()
net.add_node(1, label="Node 1") # node id = 1 and label = Node 1
net.add_node(2) # node id and label = 2

g = Network()
g.add_nodes([1,2,3], value=[10, 100, 400], title=["I am node 1", "node 2 here", "and im node 3"], x=[21.4, 54.2, 11.2], y=[100.2, 23.54, 32.1], label=["NODE 1", "NODE 2", "NODE 3"], color=["#00ff1e", "#162347", "#dd4b39"])



In [20]:
#from pyvis.network import Network
#import networkx as nx
nxg = nx.complete_graph(10)

G = Network()
G.from_nx(nxg)
#net.enable_physics(True)
net.show("mygraph.html")
